In [1]:
# Paquetes para el procesamiento de datos
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Paquetes para construir y entrenar el modelo RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, GRU, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

# Paquetes para la evaluación del modelo
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
file_path = 'DelayedFlightsClean.csv'
flights_data = pd.read_csv(file_path)


In [3]:
print(flights_data.head())
print(flights_data.columns)

   Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  ArrTime  \
0  2008      1           3          4   2003.0        1955   2211.0   
1  2008      1           3          4    754.0         735   1002.0   
2  2008      1           3          4    628.0         620    804.0   
3  2008      1           3          4   1829.0        1755   1959.0   
4  2008      1           3          4   1940.0        1915   2121.0   

   CRSArrTime UniqueCarrier  FlightNum  ... TaxiOut  Cancelled  \
0        2225            WN        335  ...     8.0      False   
1        1000            WN       3231  ...    10.0      False   
2         750            WN        448  ...    17.0      False   
3        1925            WN       3920  ...    10.0      False   
4        2110            WN        378  ...    10.0      False   

   CancellationCode  Diverted  CarrierDelay  WeatherDelay NASDelay  \
0                 N     False           0.0           0.0      0.0   
1                 N     False       

In [ ]:
# Crear la columna 'Delayed' como una variable binaria
flights_data['Delayed'] = (flights_data['ArrDelay'] > 0).astype(int)

# Verificar que se creó la columna correctamente
print(flights_data[['ArrDelay', 'Delayed']].head())

   ArrDelay  Delayed
0     -14.0        0
1       2.0        1
2      14.0        1
3      34.0        1
4      11.0        1


In [5]:
flights_data.dtypes

Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
DepTime              float64
CRSDepTime             int64
ArrTime              float64
CRSArrTime             int64
UniqueCarrier         object
FlightNum              int64
TailNum               object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin                object
Dest                  object
Distance               int64
TaxiIn               float64
TaxiOut              float64
Cancelled               bool
CancellationCode      object
Diverted                bool
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
flightDate            object
Delayed                int64
dtype: object

In [6]:
# Convertir 'flightDate' a tipo datetime
flights_data['flightDate'] = pd.to_datetime(flights_data['flightDate'], errors='coerce')

# Extraer características adicionales si es necesario (ya tenemos Month, DayofMonth, DayOfWeek)
flights_data['Year'] = flights_data['flightDate'].dt.year

In [7]:
# Función para convertir formato HHMM a horas continuas
def convert_to_hour(time):
    hour = time // 100  # Parte de las horas
    minute = time % 100  # Parte de los minutos
    return hour + minute / 60  # Convertir a una representación continua en horas

# Aplicar la función a las columnas de tiempo
flights_data['DepTime'] = flights_data['DepTime'].apply(lambda x: convert_to_hour(x) if not pd.isnull(x) else x)
flights_data['CRSDepTime'] = flights_data['CRSDepTime'].apply(lambda x: convert_to_hour(x) if not pd.isnull(x) else x)
flights_data['ArrTime'] = flights_data['ArrTime'].apply(lambda x: convert_to_hour(x) if not pd.isnull(x) else x)
flights_data['CRSArrTime'] = flights_data['CRSArrTime'].apply(lambda x: convert_to_hour(x) if not pd.isnull(x) else x)


In [8]:
# Crear variables dummy para las columnas 'UniqueCarrier', 'Origin', y 'Dest'
flights_data_encoded = pd.get_dummies(flights_data, columns=['UniqueCarrier', 'Origin', 'Dest'], drop_first=True)

# Verificar las columnas del nuevo DataFrame para confirmar que se crearon las variables dummy
print(flights_data_encoded.columns)


Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'FlightNum', 'TailNum',
       ...
       'Dest_TYR', 'Dest_TYS', 'Dest_VLD', 'Dest_VPS', 'Dest_WRG', 'Dest_WYS',
       'Dest_XNA', 'Dest_YAK', 'Dest_YKM', 'Dest_YUM'],
      dtype='object', length=650)


In [9]:
# Definir las variables seleccionadas para el modelo RNN
selected_features = [
    'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 
    'ArrTime', 'CRSArrTime', 'Distance', 'TaxiIn', 'TaxiOut'
]

# Añadir las variables dummy generadas para 'UniqueCarrier', 'Origin', y 'Dest'
# Utilizamos la lista de columnas del DataFrame codificado que empiezan con estos prefijos
selected_features += [col for col in flights_data_encoded.columns if col.startswith('UniqueCarrier_') or col.startswith('Origin_') or col.startswith('Dest_')]

# Crear el DataFrame X con las características seleccionadas y y con la variable objetivo
X = flights_data_encoded[selected_features]
y = flights_data_encoded['Delayed']

# Verificar las dimensiones de X y y para confirmar que se seleccionaron las columnas correctamente
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (1928369, 632)
Shape of y: (1928369,)


In [10]:
# Rellenar los valores nulos en el conjunto de características (X) con la mediana de cada columna
X.fillna(X.median(), inplace=True)

# Verificar si aún hay valores nulos en X
print("Valores nulos en X después de la imputación:")
print(X.isnull().sum())


C:\Users\gtgallegoswil\AppData\Local\Temp\ipykernel_21408\2496084505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.median(), inplace=True)


Valores nulos en X después de la imputación:
Month         0
DayofMonth    0
DayOfWeek     0
DepTime       0
CRSDepTime    0
             ..
Dest_WYS      0
Dest_XNA      0
Dest_YAK      0
Dest_YKM      0
Dest_YUM      0
Length: 632, dtype: int64


In [11]:
# Contar los valores en la columna 'Delayed'
delay_counts = flights_data_encoded['Delayed'].value_counts()

# Mostrar los resultados
print("Conteo de vuelos retrasados y no retrasados:")
print(delay_counts)
print("\nVuelos no retrasados (Delayed == 0):", delay_counts[0])
print("Vuelos retrasados (Delayed == 1):", delay_counts[1])

Conteo de vuelos retrasados y no retrasados:
Delayed
1    1723413
0     204956
Name: count, dtype: int64

Vuelos no retrasados (Delayed == 0): 204956
Vuelos retrasados (Delayed == 1): 1723413


In [12]:
from sklearn.utils import resample

flights_data_minority = flights_data_encoded[flights_data_encoded['Delayed'] == 0]
flights_data_majority = flights_data_encoded[flights_data_encoded['Delayed'] == 1]

# Determinar la cantidad de muestras para la clase mayoritaria (89% de la cantidad de la clase minoritaria)
n_samples_majority = int(len(flights_data_minority) * 2)  # 8 veces la clase minoritaria

# Aplicar submuestreo a la clase mayoritaria
flights_data_majority_downsampled = resample(flights_data_majority, 
                                             replace=False,             # muestreo sin reemplazo
                                             n_samples=n_samples_majority,  # número de muestras deseadas
                                             random_state=42)           # reproducibilidad

# Combinar ambas clases para crear un conjunto balanceado
flights_data_balanced = pd.concat([flights_data_minority, flights_data_majority_downsampled])

# Verificar el conteo de las clases en el conjunto balanceado
print("Conteo de clases después del resampleo:")
print(flights_data_balanced['Delayed'].value_counts())

Conteo de clases después del resampleo:
Delayed
1    409912
0    204956
Name: count, dtype: int64


In [13]:
flights_data_balanced = pd.concat([flights_data_minority, flights_data_majority_downsampled])

In [14]:
# Seleccionar las características (X) y la variable objetivo (y)
X_balanced = flights_data_balanced[selected_features]
y_balanced = flights_data_balanced['Delayed']

# Verificar las dimensiones de X_balanced y y_balanced
print("Shape of X_balanced:", X_balanced.shape)
print("Shape of y_balanced:", y_balanced.shape)


Shape of X_balanced: (614868, 632)
Shape of y_balanced: (614868,)


# Definir estructura para RNN

In [31]:
# # Tomar una muestra del 20% del conjunto balanceado
# flights_data_sampled = flights_data_balanced.sample(frac=0.2, random_state=42)

# # Convertir a tipos de datos más eficientes
# X_balanced = flights_data_sampled[selected_features].astype('float32')
# y_balanced = flights_data_sampled['Delayed']
# sequence_length = 5  # Reducir el tamaño de la secuencia

In [15]:
sequence_length = 3  # Número de vuelos en cada secuencia


In [16]:
# Convertir X_balanced y y_balanced a numpy para manipulación más fácil
X_values = X_balanced.values
y_values = y_balanced.values

# Listas para almacenar las secuencias
X_sequences = []
y_sequences = []

# Crear las secuencias
for i in range(len(X_values) - sequence_length):
    # Crear una secuencia de tamaño 'sequence_length' para X
    X_seq = X_values[i:i + sequence_length]
    X_sequences.append(X_seq)
    
    # La etiqueta (y) es el valor que sigue a la secuencia actual
    y_sequences.append(y_values[i + sequence_length])

# Convertir las listas a arreglos numpy
X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

# Verificar las dimensiones de las secuencias generadas
print("Shape of X_sequences:", X_sequences.shape)
print("Shape of y_sequences:", y_sequences.shape)


Shape of X_sequences: (614865, 3, 632)
Shape of y_sequences: (614865,)
